<img src="./images/SMILESX_logo.png" alt="drawing" width="300" align="left"/>

# Welcome to the SMILES-X tutorial!

In this brief tutorial we cover the basics of the SMILES-X package.
Here we show how to train, extract and use trained SMILES-X models. More detailed information on the package implementation and fundamentals can be found in the documentation.

The main body of the SMILES-X is contained in the `main.py`, which we need to import.\We will also need to import the library that is going to be used for data reading and preprocessing. Here we use `pandas` to work with the `./data/example_data.csv` data file.

In [ ]:
import pandas as pd

from SMILESX import main

%load_ext autoreload
%aimport SMILESX
%autoreload 1
%matplotlib inline

In [ ]:
# Define data folder and extension
data_path = "./data/example_data.csv"

In [ ]:
# Provide data information
data_name = 'Test'
data_label = 'Test label' # will show on plots
data_units = 'units' # will show on plots

In [ ]:
# Load the data
data = pd.read_csv(data_path)

## Data format

The data format is not critical as long as it can be shaped into a numpy array, and provides the following information:
- SMILES
- ground true values (experimental or theoretical, mean or median)


It can **optionally** include:
- minimum and maximum values (error bars will represent the range of values)
- standard deviation (error bars will represent one standard deviation)
- additional numeric values

In [ ]:
data.head(10)

Our example data file consists of real polymeric SMILES, and is accompanied by fake median, mininum and maximum values. \
It also contains auxiliary data `extra_1`, `extra_2`, `extra_3`.

## Hyperparameters optimization

First, we have to optimize the architecture shape as well as its hyperparameters. For this, we have to define the hyperparameters space. In the SMILES-X, the hyperparameters that can be optimized are:

**Geometry-related hyperparameters**
- Embedding size\
    The dimensionality of the embedding
- LSTM layer size\
    Number of units in the LSTM layer. As we use bidirectional LSTM, this number is internally multiplied by two during the run.
- Time-distributed (TD) dense layer size\
    Number of units in the TD dense layer

**Training-related hyperparameters**
- Batch size
- Learning rate
   
For each of the hyperparameter the bounds are defined as a **list of discrete values**.\
For the learning rate, we give a range of powers of 10.\
The typical bounds that can be used:

    embed_bounds = [8, 16, 32, 64, 128, 256, 512]
    lstm_bounds = [8, 16, 32, 64, 128, 256, 512]
    tdense_bounds = [8, 16, 32, 64, 128, 256, 512]
    bs_bounds = [8, 16, 32, 64, 128, 256, 512]
    lr_bounds = [2, 2.1, 2.2, ..., 3.8, 3.9]
  
Note, that for the sake of demonstration, we run the code with narrower bounds:

In [ ]:
# Defining the hyperparameters' bounds
embed_bounds = [8, 16] # embedding size
lstm_bounds = [8, 16] # number of units in the LSTM layer
tdense_bounds = [8, 16] # number of units in the dense layer
bs_bounds = [64, 128] # batch size
lr_bounds = [2., 2.5, 3., 3.5] # learning rate

Depending on the user request, hyperparameters optimization can be performed in single- or two-steps fashion.

Single-step optimizations implies optimization of all the given hyperparameters simultaneously via Bayesian optimization. Bayesian optimization is activate by setting `bayopt_mode='on'`.

For the two-step optimization, SMILES-X implements zero-cost optimization of geometry-related hyperparameters. This allows to scan through the whole set of possible combinations of `[embed_bounds, lstm_bounds, tdense_bounds]` in a relatively short period of time. Trainless geometry optimization can be enable by setting `geomopt_mode='on'`. The remaining training-related hyperparameters are then optimized via Bayesian optimization, if it is activated. As trainless geometry optimization provides good and stable results, the two-step optimization is set by default.

Hyperparameters optimization can be skipped altogether by setting both `bayopt_mode` and `geomopt_mode` to `'off'`. In this case, the user should provide referense values of the hyperparameters (`embed_ref`, `lstm_ref`, `dense_ref`, `bs_ref` and `lr_ref`). In case where no reference values are provided, default values are used. 

Note, that only the hyperparameters optimization will only optimize those hyperparameters, which have defined bounds, in accordance with the requested modes. The remaining onse will be set either to the user-defined or to the default values. For instance, it is possible to optimize only a part of geometry, or only the learning rate.


## Training

The SMILES-X training is performed by calling the `main` function. Here we define the inputs, optimization parameters, training parameters (number of epochs, number of independent runs, number of folds in k-fold cross validation, learning rate schedule and so on) and GPU setup.

This will create a new path in the `./outputs` directory, corresponding to the indicated `data_name`. All the models, as well as auxiliary data such as scalers and logs, are stored there.

Now you are ready to run the first SMILES-X model training!

In [ ]:
main.main(data_smiles=data[['smiles1','smiles2']], # SMILES input
          data_prop=data['median'], # Property of interest
          data_err=data[['min', 'max']], # Error on the property
          data_name=data_name,
          data_units=data_units,
          data_label=data_label,
          smiles_concat=True,
          geomopt_mode='off', # Zero-cost geometry optimization
          bayopt_mode='off', # Bayesian optimization
          train_mode='on', # Train
          bs_bounds=bs_bounds,
          lr_bounds=lr_bounds,
          embed_bounds=embed_bounds,
          lstm_bounds=lstm_bounds,
          tdense_bounds=tdense_bounds,
          k_fold_number=2, # Number of cross-validation splits
          n_runs=3,# Number of runs per fold
          check_smiles=True, # Verify SMILES validity via RDKit
          augmentation=True, # Augment the data or not
          bayopt_n_rounds=2,
          bayopt_n_epochs=5,
          bayopt_n_runs=2,
          n_gpus=1,
          n_epochs=10,
          log_verbose=True, # To send print outs both to the file and console
          train_verbose=True) # Show model training progress)

The output data is organized in the following way:

```
./outputs
└───Test
    └───Augm
        └───Train
            │   Scores_Folds.csv # RMSE, MAE, R2 scores per fold
            │   Scores_Final.csv # Final out-of-sample RMSE, MAE, R2 scores
            │   Predictions.csv # Out-of-samples predictions for each SMILES
            │   Train_yyyy-mm-dd_hh:mm:ss.log # Logs
            │
            └───Plots # Prediction vs Observation plots per fold and final out-of-sample
            │   └───Prediction_vs_Observation
            │   │   │   Test_PredvsTrue_Plot.png
            │   │   │
            │   │   └───Runs
            │   │   │       Test_PredvsTrue_Plot_Fold_0_Run_0.png
            │   │   │       Test_PredvsTrue_Plot_Fold_0_Run_1.png
            │   │   │       Test_PredvsTrue_Plot_Fold_0_Run_2.png
            │   │   │       Test_PredvsTrue_Plot_Fold_1_Run_0.png
            │   │   │       ...
            │   │   │
            │   │   └───Folds
            │   │           Test_PredvsTrue_Plot_Fold_0.png
            │   │           Test_PredvsTrue_Plot_Fold_1.png
            │   │           Test_PredvsTrue_Plot_Fold_2.png
            │   │
            │   └───Learning_Curves # Learning curves, one per run
            │           Test_FitHistory_Fold_0_Run_0.png
            │           Test_FitHistory_Fold_0_Run_1.png
            │           Test_FitHistory_Fold_0_Run_2.png
            │           Test_FitHistory_Fold_1_Run_0.png
            │           ...
            │
            └───Models # One per run
            │   │   Test_Model_Fold_0_Run_0.hdf5 
            │   │   Test_Model_Fold_0_Run_1.hdf5 
            │   │   Test_Model_Fold_0_Run_2.hdf5 
            │   │   Test_Model_Fold_1_Run_0.hdf5
            │   │   ...
            │
            └───Other # Auxiliary files
                │   Test_Hyperparameters.csv # Hyperparameters used for training
                │   Test_GeomScores.csv # Geometry optimization scores, sorted
                │   Test_Vocabulary.txt # Vocabulary
                │
                └───Scalers
                    │   Test_Scaler_Extra.pkl # Scaler for additional data input (if `data_extra` is provided)
                    │   Test_Scaler_fold_0.pkl # Scaler for the outputs, one per fold
                    │   Test_Scaler_fold_1.pkl # Scaler for the outputs, one per fold
                    │   Test_Scaler_fold_2.pkl # Scaler for the outputs, one per fold
```
First, `Test` directory is created in `./outputs/`, which corresponds to the provided `data_name`. As the data augmentation has been requested, an `Augm` directory is created whithin `Test` one (otherwise it would be called `Can` for "canonical"). Finally, all the output files related to training are stored within the `Train` directory. 

First of, this is where per-SMILES predictions (`*_Predictions.csv`) and per-fold and final out-of-sample statistics (`*_Scores_Folds.csv` and `*_Scores_Final.csv`) are stored. The following several lines help to access them:

In [ ]:
from tabulate import tabulate
predictions = pd.read_csv("./outputs/{0}/Augm/Train/{0}_Predictions.csv".format(data_name))
scores_folds = pd.read_csv("./outputs/{0}/Augm/Train/{0}_Scores_Folds.csv".format(data_name), header=[0,1,2],index_col=[0])
scores_final = pd.read_csv("./outputs/{0}/Augm/Train/{0}_Scores_Final.csv".format(data_name), header=[0,1])

In [ ]:
predictions

In [ ]:
scores_folds

In [ ]:
scores_final

## Additional numerical inputs

This time we are going to traing a model with some additional data. These data will be injected in the model at the level between the attention and the output layer. In order to add non-linearity between the additional data, we insert 3 additional dense layers. We do not perform geometry optimization and set the architecture to some reference values. Data augmentation is set to `False`.

In [ ]:
main.main(data_smiles=data['smiles'], # SMILES input
          data_prop=data['median'], # Property of interest
          data_err=data[['min', 'max']], # Error on the property
          data_extra=data[['extra_1', 'extra_2', 'extra_3']], # Auxiliary input data
          data_name=data_name+'_EXTRA',
          data_units=data_units,
          data_label=data_label,
          geomopt_mode='off', # Zero-cost geometry optimization
          bayopt_mode='off', # Bayesian optimization
          train_mode='on', # Train
          bs_ref=128,
          lr_ref=3.0,
          embed_ref=8,
          lstm_ref=8,
          tdense_ref=8,
          dense_depth=3,
          k_fold_number=2, # Number of cross-validation splits
          n_runs=3,# Number of runs per fold
          check_smiles=True, # Verify SMILES validity via RDKit
          augmentation=False, # Augment the data (only when `check_smiles` is set to `True`)
          bayopt_n_rounds=2,
          bayopt_n_epochs=5,
          bayopt_n_runs=2,
          n_gpus=1,
          n_epochs=10,
          log_verbose=True, # To send print outs both to the file and console
          train_verbose=False) # Show model training progress)

Notice the difference in the model structure printed by Keras.

The outputs can be found in the `./outputs/Test_EXTRA/Can/Train/` directory.

# How to use models

Once the training is finished we have the access to the ensemble of models via `loadmodel` module. All that should be specified is the data_name and the augmentation flag used for training. The module will automatically collect models for each fold and run and return the ensemble together with auxiliary information, such as scalers or dataset statistics.

## Load model ensemble

First, we will load the model for the simple version where the model is trained only with SMILES (no additional data).

In [ ]:
from SMILESX import loadmodel

# Load trained models once, use as many times as needed
model = loadmodel.LoadModel(data_name = data_name,
                            augment = True,
                            gpu_ind = 0,
                            return_attention = True)

Then, the loaded model instance can be used for inference and/or interpretation. 

### Inference

In [ ]:
from SMILESX import inference

preds = inference.infer(model=model,
                        data_smiles=['*OC(C)C(=O)*','CCC','C=O'],
                        augment=True,
                        check_smiles=True,
                        log_verbose=True)

##### In case of a classication task
- Apply a threshold in order to reduce the predicted values to binary classes

In [ ]:
preds_asclass = (preds['mean'] > 0.5).astype("int8")

### Interpretation

In [ ]:
from SMILESX import interpret

inter = interpret.interpret(model=model,
                            smiles=preds['SMILES'],
                            pred=preds[['mean', 'sigma']],
                            check_smiles=True,
                            log_verbose=True)

Let us load models containing additional inputs. As the model is adjusted to take the additional inputs, they are also requested at the time of inference. For the same reason, temporal distance plot is not available for the models with additional inputs.

In [ ]:
model = loadmodel.LoadModel(data_name = data_name+'_EXTRA',
                            augment = False, # Since the model was trained without augmentation
                            gpu_ind = 0,
                            return_attention = True)

Note that data augmentation at the inference time can be requested regardless whether it was used for training or not. In this case, mean and standard deviation are computed over the augmentations and model ensemble.

In [ ]:
preds = inference.infer(model=model,
                        data_smiles=['*OC(C)C(=O)*','CCC','C=O'],
                        data_extra=[[150,0.5,4], [87,0.2,15], [54,1.2,24]],
                        augment=True, # Since we want to augment the data before making inference
                        check_smiles=True,
                        log_verbose=True)

Let us create fake true values for the molecules we passed for the prediction, to see the values on the interpretation 2D figures.

In [ ]:
import pandas as pd
# Fake ground truth values to show them 2D interpretation plots
trues = pd.DataFrame([[400, 370, 415],
                      [300, 280, 310],
                      [200, 190, 205]])
trues.columns = ['median', 'min', 'max']

Note the absence of temporal difference plots.

In [ ]:
inter = interpret.interpret(model=model,
                            smiles=preds['SMILES'],
                            pred=preds[['mean', 'sigma']],
                            true=trues['median'],
                            true_err=trues[['min','max']],
                            check_smiles=True,
                            log_verbose=True)